In [1]:
%load_ext autoreload
%autoreload 2

import gradio

In [2]:
def test(x):
    return x.upper()

In [3]:
iface = gradio.Interface(input="textbox", output="textbox", model=test)

Model type not explicitly identified, inferred to be: python function


In [4]:
iface.launch()

NOTE: Gradio is in beta2 stage, please report all bugs to: a12d@stanford.edu
Model available locally at: http://localhost:7861/interface.html
To create a public link, set `share_link=True` in the argument to `launch()`


127.0.0.1 - - [24/Feb/2019 22:21:36] "GET /interface.html HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:36] "GET /js/all-io.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:36] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:37] "GET /css/draw-a-digit.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:37] "GET /js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:37] "GET /js/bootstrap-notify.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:37] "GET /js/textbox-output.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2019 22:21:37] "GET /js/textbox-input.js HTTP/1.1" 200 -
